In [8]:
# https://www.youtube.com/watch?v=i0oPFKrADTo&ab_channel=PyData

import pandas as pd
from sklearn import model_selection, feature_extraction, naive_bayes, metrics, ensemble, preprocessing
import xgboost

In [3]:
import sys
import os
workspace_path = os.path.join(os.path.abspath(".."))
sys.path.append(workspace_path)
filepath = os.path.join(workspace_path, "data", "cleaned_data", "bulletins_w_labels_and_content.csv")
df = pd.read_csv(filepath)
df

,Unnamed: 0.1,Unnamed: 0,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,File Names,Label,Text,Cleaned text
0,0,0,9206,311 DIRECTOR,54,20,31,3,311 DIRECTOR 9206 041814.txt,M,311 DIRECTOR Class Code: 9206 Open Date:...,director class code open date annual salary du...
1,1,1,1223,ACCOUNTING CLERK,648,488,152,8,ACCOUNTING CLERK 1223 071318.txt,W,ACCOUNTING CLERK Class Code: 1223 Open ...,accounting clerk class code open date exam ope...
2,2,2,7260,AIRPORT MANAGER,51,13,37,1,AIRPORT MANAGER 7260 120216.txt,M,AIRPORT MANAGER Class Code: 7260 Open D...,airport manager class code open date exam open...
3,3,3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,AIRPORT POLICE LIEUTENANT 3227 091616.txt,M,AIRPORT POLICE LIEUTENANT ...,airport police lieutenant class code open date...
4,4,4,2400,AQUARIST,40,15,24,1,AQUARIST 2400 050214.txt,M,AQUARIST Class Code: 2400 Open Date: 05...,aquarist class code open date annual salary ca...
...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,WASTEWATER TREATMENT LABORATORY MANAGER 7840 1...,M,WASTEWATER TREATMENT LABORATORY MANAGER Class...,wastewater treatment laboratory manager class ...
173,173,173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,WASTEWATER TREATMENT OPERATOR 120718.txt,M,WASTEWATER TREATMENT OPERATOR Class Code: ...,wastewater treatment operator class code open ...
174,174,174,7857,WATER MICROBIOLOGIST,179,89,82,8,WATER MICROBIOLOGIST 7857 072514 rev073114.txt,N,WATER MICROBIOLOGIST Class Code: 7857...,water microbiologist class code open date revi...
175,175,175,3912,WATER UTILITY WORKER,96,2,92,2,WATER UTILITY WORKER 3912 120817.txt,M,WATER UTILITY WORKER Class Code: 3912 Op...,water utility worker class code open date exam...


In [5]:
# Create and add labels
mostly_women = df["Female"] >= (df["Apps Received"] - df["Unknown_Gender"]) * 0.7
mostly_men = df["Male"] >= (df["Apps Received"] - df["Unknown_Gender"]) * 0.7
neutral = (df["Female"] < (df["Apps Received"] - df["Unknown_Gender"]) * 0.7) & (df["Female"] > (df["Apps Received"] - df["Unknown_Gender"]) * 0.3)

In [4]:
df = df.drop("Label", axis=1)
df

,Unnamed: 0.1,Unnamed: 0,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,File Names,Text,Cleaned text
0,0,0,9206,311 DIRECTOR,54,20,31,3,311 DIRECTOR 9206 041814.txt,311 DIRECTOR Class Code: 9206 Open Date:...,director class code open date annual salary du...
1,1,1,1223,ACCOUNTING CLERK,648,488,152,8,ACCOUNTING CLERK 1223 071318.txt,ACCOUNTING CLERK Class Code: 1223 Open ...,accounting clerk class code open date exam ope...
2,2,2,7260,AIRPORT MANAGER,51,13,37,1,AIRPORT MANAGER 7260 120216.txt,AIRPORT MANAGER Class Code: 7260 Open D...,airport manager class code open date exam open...
3,3,3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,AIRPORT POLICE LIEUTENANT 3227 091616.txt,AIRPORT POLICE LIEUTENANT ...,airport police lieutenant class code open date...
4,4,4,2400,AQUARIST,40,15,24,1,AQUARIST 2400 050214.txt,AQUARIST Class Code: 2400 Open Date: 05...,aquarist class code open date annual salary ca...
...,...,...,...,...,...,...,...,...,...,...,...
172,172,172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,WASTEWATER TREATMENT LABORATORY MANAGER 7840 1...,WASTEWATER TREATMENT LABORATORY MANAGER Class...,wastewater treatment laboratory manager class ...
173,173,173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,WASTEWATER TREATMENT OPERATOR 120718.txt,WASTEWATER TREATMENT OPERATOR Class Code: ...,wastewater treatment operator class code open ...
174,174,174,7857,WATER MICROBIOLOGIST,179,89,82,8,WATER MICROBIOLOGIST 7857 072514 rev073114.txt,WATER MICROBIOLOGIST Class Code: 7857...,water microbiologist class code open date revi...
175,175,175,3912,WATER UTILITY WORKER,96,2,92,2,WATER UTILITY WORKER 3912 120817.txt,WATER UTILITY WORKER Class Code: 3912 Op...,water utility worker class code open date exam...


In [6]:
labels = []
for i in range(len(df)):
    if mostly_women[i]:
        label = 2
    elif mostly_men[i]:
        label = 0
    elif neutral[i]:
        label = 1
    else:
        label = None
    labels.append(label)
df["Label"] = labels
df

,Unnamed: 0.1,Unnamed: 0,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,File Names,Text,Cleaned text,Label
0,0,0,9206,311 DIRECTOR,54,20,31,3,311 DIRECTOR 9206 041814.txt,311 DIRECTOR Class Code: 9206 Open Date:...,director class code open date annual salary du...,1
1,1,1,1223,ACCOUNTING CLERK,648,488,152,8,ACCOUNTING CLERK 1223 071318.txt,ACCOUNTING CLERK Class Code: 1223 Open ...,accounting clerk class code open date exam ope...,2
2,2,2,7260,AIRPORT MANAGER,51,13,37,1,AIRPORT MANAGER 7260 120216.txt,AIRPORT MANAGER Class Code: 7260 Open D...,airport manager class code open date exam open...,0
3,3,3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,AIRPORT POLICE LIEUTENANT 3227 091616.txt,AIRPORT POLICE LIEUTENANT ...,airport police lieutenant class code open date...,0
4,4,4,2400,AQUARIST,40,15,24,1,AQUARIST 2400 050214.txt,AQUARIST Class Code: 2400 Open Date: 05...,aquarist class code open date annual salary ca...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,WASTEWATER TREATMENT LABORATORY MANAGER 7840 1...,WASTEWATER TREATMENT LABORATORY MANAGER Class...,wastewater treatment laboratory manager class ...,1
173,173,173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,WASTEWATER TREATMENT OPERATOR 120718.txt,WASTEWATER TREATMENT OPERATOR Class Code: ...,wastewater treatment operator class code open ...,0
174,174,174,7857,WATER MICROBIOLOGIST,179,89,82,8,WATER MICROBIOLOGIST 7857 072514 rev073114.txt,WATER MICROBIOLOGIST Class Code: 7857...,water microbiologist class code open date revi...,1
175,175,175,3912,WATER UTILITY WORKER,96,2,92,2,WATER UTILITY WORKER 3912 120817.txt,WATER UTILITY WORKER Class Code: 3912 Op...,water utility worker class code open date exam...,0


In [7]:
X = df["Cleaned text"]
y = df["Label"]

In [9]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

vectorizer = feature_extraction.text.CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [10]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=45, stratify=y)

In [11]:
mnb = naive_bayes.MultinomialNB()
mnb.fit(X_train, y_train)
prediction = mnb.predict(X_test)
matrix = metrics.classification_report(y_test, prediction)

c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
print(matrix)

              precision    recall  f1-score   support

           0       0.92      0.81      0.86        27
           1       0.71      0.94      0.81        16
           2       0.00      0.00      0.00         2

    accuracy                           0.82        45
   macro avg       0.54      0.58      0.56        45
weighted avg       0.80      0.82      0.81        45



In [13]:
from xgboost import XGBClassifier

mxgb = XGBClassifier(use_label_encoder=False)
mxgb.fit(X_train, y_train)
prediction = mnb.predict(X_test)
matrix = metrics.classification_report(y_test, prediction)

c:\Code\job_discrimination\venv\lib\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and bein

In [14]:
print(matrix)

              precision    recall  f1-score   support

           0       0.92      0.81      0.86        27
           1       0.71      0.94      0.81        16
           2       0.00      0.00      0.00         2

    accuracy                           0.82        45
   macro avg       0.54      0.58      0.56        45
weighted avg       0.80      0.82      0.81        45



In [18]:
kf = model_selection.StratifiedKFold(n_splits=10, shuffle=False, random_state=None)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    mxgb.fit(X_train, y_train)
    prediction = mxgb.predict(X_test)
    matrix = metrics.classification_report(y_test, prediction)
    print(matrix)

c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.80      0.80      0.80        10
           1       0.75      0.86      0.80         7
           2       0.00      0.00      0.00         1

    accuracy                           0.78        18
   macro avg       0.52      0.55      0.53        18
weighted avg       0.74      0.78      0.76        18



c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.64      0.90      0.75        10
           1       0.75      0.43      0.55         7
           2       0.00      0.00      0.00         1

    accuracy                           0.67        18
   macro avg       0.46      0.44      0.43        18
weighted avg       0.65      0.67      0.63        18



c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.71      1.00      0.83        10
           1       1.00      0.50      0.67         6
           2       0.00      0.00      0.00         1

    accuracy                           0.76        17
   macro avg       0.57      0.50      0.50        17
weighted avg       0.77      0.76      0.73        17



c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.64      0.70      0.67        10
           1       0.50      0.50      0.50         6
           2       0.00      0.00      0.00         1

    accuracy                           0.59        17
   macro avg       0.38      0.40      0.39        17
weighted avg       0.55      0.59      0.57        17

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        10
           1       1.00      1.00      1.00         6
           2       0.00      0.00      0.00         1

    accuracy                           0.94        17
   macro avg       0.64      0.67      0.65        17
weighted avg       0.89      0.94      0.91        17



c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Code\job_discrimination\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
